<a href="https://colab.research.google.com/github/RM-RAMASAMY/LOW_CODE_AUTOML_Autogluon/blob/main/AutoGluon_for_Kaggle_competitions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!python -m pip install --upgrade pip &>/dev/null
!python -m pip install autogluon &>/dev/null

In [2]:
!pip install kaggle
!mkdir -p ~/.kaggle
!echo '{"username":"rmramasamy","key":"3125143e12175deb565fc3ebb685c8d6"}' > ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!kaggle competitions download -c ieee-fraud-detection  --force

 97% 115M/118M [00:00<00:00, 259MB/s] 
100% 118M/118M [00:00<00:00, 198MB/s]


In [4]:
import pandas as pd
import numpy as np
from autogluon.tabular import TabularPredictor

directory = '/content/IEEEfraud/'  # directory where you have downloaded the data CSV files from the competition
label = 'isFraud'  # name of target variable to predict in this competition
eval_metric = 'roc_auc'  # Optional: specify that competition evaluation metric is AUC
save_path = directory + 'AutoGluonModels/'  # where to store trained models



import zipfile

with zipfile.ZipFile("/content/ieee-fraud-detection.zip", 'r') as zip_ref:
    zip_ref.extractall("/content/IEEEfraud")

In [44]:
train_identity = pd.read_csv(directory+'train_identity.csv')
train_transaction = pd.read_csv(directory+'train_transaction.csv')

In [45]:
train_data = pd.merge(train_transaction, train_identity, on='TransactionID', how='left')

In [46]:
predictor = TabularPredictor(label=label, eval_metric=eval_metric, path=save_path, verbosity=3).fit(
    train_data, presets='optimize_for_deployment', time_limit=60,
)

results = predictor.fit_summary();

Verbosity: 3 (Detailed Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Thu Jun 27 21:05:47 UTC 2024
CPU Count:          96
GPU Count:          0
Memory Avail:       309.95 GB / 334.56 GB (92.6%)
Disk Space Avail:   196.37 GB / 225.33 GB (87.1%)
Presets specified: ['optimize_for_deployment']
============ fit kwarg info ============
User Specified kwargs:
{'keep_only_best': True, 'save_space': True}
Full kwargs:
{'_feature_generator_kwargs': None,
 '_save_bag_folds': None,
 'ag_args': None,
 'ag_args_ensemble': None,
 'ag_args_fit': None,
 'auto_stack': False,
 'calibrate': 'auto',
 'ds_args': {'clean_up_fits': True,
             'detection_time_frac': 0.25,
             'enable_ray_logging': True,
             'holdout_data': None,
             'holdout_frac': 0.1111111111111111,
             'memory_safe_fits': True,
        

*** Summary of fit() ***
Estimated performance of each model:
                 model  score_val eval_metric  pred_time_val   fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0  WeightedEnsemble_L2   0.830415     roc_auc       3.516271  18.407926                0.000998           0.058919            2       True          3
1           LightGBMXT   0.808376     roc_auc       0.023974   8.936550                0.023974           8.936550            1       True          2
2       KNeighborsUnif   0.671934     roc_auc       3.491299   9.412457                3.491299           9.412457            1       True          1
Number of models trained: 3
Types of models trained:
{'KNNModel', 'WeightedEnsembleModel', 'LGBModel'}
Bagging used: False 
Multi-layer stack-ensembling used: False 
Feature Metadata (Processed):
(raw dtype, special dtypes):
('category', []) :  31 | ['ProductCD', 'card4', 'card6', 'P_emaildomain', 'R_emaildomain', ...]
('float', [])    

/usr/local/lib/python3.10/dist-packages/autogluon/core/utils/plots.py:169: UserWarning: AutoGluon summary plots cannot be created because bokeh is not installed. To see plots, please do: "pip install bokeh==2.0.1"
  warnings.warn('AutoGluon summary plots cannot be created because bokeh is not installed. To see plots, please do: "pip install bokeh==2.0.1"')


In [47]:
test_identity = pd.read_csv(directory+'test_identity.csv')
test_transaction = pd.read_csv(directory+'test_transaction.csv')
# Get the column names from train_identity
train_identity_columns = train_identity.columns

# Rename the columns in test_identity
test_identity.columns = train_identity_columns

test_data = pd.merge(test_transaction, test_identity, on='TransactionID', how='left')  # same join applied to training files

In [48]:
y_predproba = predictor.predict_proba(test_data,)
y_predproba.head(5)  # some example predicted fraud-probabilities

Loading: /content/IEEEfraud/AutoGluonModels/models/KNeighborsUnif/model.pkl
Loading: /content/IEEEfraud/AutoGluonModels/models/LightGBMXT/model.pkl
Loading: /content/IEEEfraud/AutoGluonModels/models/WeightedEnsemble_L2/model.pkl


,0,1
0,0.970004,0.029996
1,0.970004,0.029996
2,0.970004,0.029996
3,0.970412,0.029588
4,0.970412,0.029588


In [49]:
predictor.positive_class

1

In [50]:
predictor.class_labels  # classes in this list correspond to columns of predict_proba() output

[0, 1]

In [52]:
submission = pd.read_csv(directory+'sample_submission.csv')
submission['isFraud'] = y_predproba[1]
submission.head()
submission.to_csv(directory+'my_submission.csv', index=False)

In [54]:
!kaggle competitions submit -c ieee-fraud-detection -f /content/IEEEfraud/sample_submission.csv -m "my first submission"

100% 5.80M/5.80M [00:00<00:00, 12.5MB/s]
Successfully submitted to IEEE-CIS Fraud Detection